In [1]:
import torch
from tqdm import tqdm
import time
from contextlib import nullcontext
import random
import numpy as np

In [2]:
sentences = [] 
with open("output_data.txt","r") as f:
    sentences = f.readlines()

In [3]:
sentences[:10]

['to have him suddenly_ylneddus mih evah ot^\n',
 ' drovethe bristled _ deltsirb ehtevord ^\n',
 'e to thy bridal cham_mahc ladirb yht ot e^\n',
 'd infidels,and in t_t ni dna,sledifni d^\n',
 'her most vile princi_icnirp eliv tsom reh^\n',
 'es upon her peaceful_lufecaep reh nopu se^\n',
 ' marriage: when and _ dna nehw :egairram ^\n',
 'brother gloucester h_h retsecuolg rehtorb^\n',
 'alks, my manors that_taht sronam ym ,skla^\n',
 'e bestow a simple in_ni elpmis a wotseb e^\n']

In [15]:
# at beginning of the script
device_type = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device_type)
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScale

cuda:0


In [16]:
total_vocab = sorted(list(set("".join(sentences))))
vocab_size = len(total_vocab)

In [17]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(total_vocab) }
itos = { i:ch for i,ch in enumerate(total_vocab) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there^"))
print(decode(encode("hii there")))

[22, 23, 23, 1, 34, 22, 19, 32, 19, 13]
hii there


In [18]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = []
for line in tqdm(sentences): 
    data.append(torch.tensor(encode(line), dtype=torch.long))
print(data[:5])

100%|██████████| 1000000/1000000 [00:11<00:00, 87391.98it/s]

[tensor([34, 29,  1, 22, 15, 36, 19,  1, 22, 23, 27,  1, 33, 35, 18, 18, 19, 28,
        26, 39, 14, 39, 26, 28, 19, 18, 18, 35, 33,  1, 27, 23, 22,  1, 19, 36,
        15, 22,  1, 29, 34, 13,  0]), tensor([ 1, 18, 32, 29, 36, 19, 34, 22, 19,  1, 16, 32, 23, 33, 34, 26, 19, 18,
         1, 14,  1, 18, 19, 26, 34, 33, 23, 32, 16,  1, 19, 22, 34, 19, 36, 29,
        32, 18,  1, 13,  0]), tensor([19,  1, 34, 29,  1, 34, 22, 39,  1, 16, 32, 23, 18, 15, 26,  1, 17, 22,
        15, 27, 14, 27, 15, 22, 17,  1, 26, 15, 18, 23, 32, 16,  1, 39, 22, 34,
         1, 29, 34,  1, 19, 13,  0]), tensor([18,  1, 23, 28, 20, 23, 18, 19, 26, 33,  6, 15, 28, 18,  1, 23, 28,  1,
        34, 14, 34,  1, 28, 23,  1, 18, 28, 15,  6, 33, 26, 19, 18, 23, 20, 28,
        23,  1, 18, 13,  0]), tensor([22, 19, 32,  1, 27, 29, 33, 34,  1, 36, 23, 26, 19,  1, 30, 32, 23, 28,
        17, 23, 14, 23, 17, 28, 23, 32, 30,  1, 19, 26, 23, 36,  1, 34, 33, 29,
        27,  1, 32, 19, 22, 13,  0])]


In [19]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [20]:
block_size = 20

x = train_data[1][:block_size]
y = train_data[1][1:block_size+1]
print(x, y)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

tensor([ 1, 18, 32, 29, 36, 19, 34, 22, 19,  1, 16, 32, 23, 33, 34, 26, 19, 18,
         1, 14]) tensor([18, 32, 29, 36, 19, 34, 22, 19,  1, 16, 32, 23, 33, 34, 26, 19, 18,  1,
        14,  1])
when input is tensor([1]) the target: 18
when input is tensor([ 1, 18]) the target: 32
when input is tensor([ 1, 18, 32]) the target: 29
when input is tensor([ 1, 18, 32, 29]) the target: 36
when input is tensor([ 1, 18, 32, 29, 36]) the target: 19
when input is tensor([ 1, 18, 32, 29, 36, 19]) the target: 34
when input is tensor([ 1, 18, 32, 29, 36, 19, 34]) the target: 22
when input is tensor([ 1, 18, 32, 29, 36, 19, 34, 22]) the target: 19
when input is tensor([ 1, 18, 32, 29, 36, 19, 34, 22, 19]) the target: 1
when input is tensor([ 1, 18, 32, 29, 36, 19, 34, 22, 19,  1]) the target: 16
when input is tensor([ 1, 18, 32, 29, 36, 19, 34, 22, 19,  1, 16]) the target: 32
when input is tensor([ 1, 18, 32, 29, 36, 19, 34, 22, 19,  1, 16, 32]) the target: 23
when input is tensor([ 1, 18, 32, 29, 36

In [21]:
torch.randint(10, (32,))

tensor([5, 9, 3, 4, 9, 6, 2, 0, 6, 2, 7, 9, 7, 3, 3, 4, 3, 7, 0, 9, 0, 9, 6, 9,
        5, 4, 8, 8, 6, 0, 0, 0])

In [23]:
torch.manual_seed(42)
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 16 # what is the maximum context length for predictions?

# poor man's data loader
def get_batch(sample):
    data = sample
    ix = torch.randint(len(data) - block_size + 1, (batch_size,))
    x_temps = [data[i:i+block_size] for i in ix]
    x = []
    for _x in x_temps:
        if len(_x) < block_size:
            while len(_x) < block_size:
                new_element = torch.tensor(encode("^"))
                _x = torch.cat((_x,new_element),0)
        x.append(_x)
    
    y_temps = [data[i+1:i+1+block_size] for i in ix]
    y = []
    for _y in y_temps:
        
        if len(_y) < block_size:
            while len(_y) < block_size:
                new_element = torch.tensor(encode("^"))
                _y = torch.cat((_y,new_element),0)
        y.append(_y)
    
    x = torch.stack(x)
    y = torch.stack(y)
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device_type, non_blocking=True), y.pin_memory().to(device_type, non_blocking=True)
    else:
        x, y = x.to(device_type), y.to(device_type)
    return x, y

In [24]:
train_data[0]

tensor([34, 29,  1, 22, 15, 36, 19,  1, 22, 23, 27,  1, 33, 35, 18, 18, 19, 28,
        26, 39, 14, 39, 26, 28, 19, 18, 18, 35, 33,  1, 27, 23, 22,  1, 19, 36,
        15, 22,  1, 29, 34, 13,  0])

In [26]:
get_batch(train_data[0])

(tensor([[26, 28, 19, 18, 18, 35, 33,  1, 27, 23, 22,  1, 19, 36, 15, 22],
         [28, 19, 18, 18, 35, 33,  1, 27, 23, 22,  1, 19, 36, 15, 22,  1],
         [15, 36, 19,  1, 22, 23, 27,  1, 33, 35, 18, 18, 19, 28, 26, 39],
         [18, 35, 33,  1, 27, 23, 22,  1, 19, 36, 15, 22,  1, 29, 34, 13],
         [26, 39, 14, 39, 26, 28, 19, 18, 18, 35, 33,  1, 27, 23, 22,  1],
         [22, 15, 36, 19,  1, 22, 23, 27,  1, 33, 35, 18, 18, 19, 28, 26],
         [33, 35, 18, 18, 19, 28, 26, 39, 14, 39, 26, 28, 19, 18, 18, 35],
         [33, 35, 18, 18, 19, 28, 26, 39, 14, 39, 26, 28, 19, 18, 18, 35],
         [18, 18, 19, 28, 26, 39, 14, 39, 26, 28, 19, 18, 18, 35, 33,  1],
         [39, 26, 28, 19, 18, 18, 35, 33,  1, 27, 23, 22,  1, 19, 36, 15],
         [18, 18, 19, 28, 26, 39, 14, 39, 26, 28, 19, 18, 18, 35, 33,  1],
         [18, 35, 33,  1, 27, 23, 22,  1, 19, 36, 15, 22,  1, 29, 34, 13],
         [19,  1, 22, 23, 27,  1, 33, 35, 18, 18, 19, 28, 26, 39, 14, 39],
         [26, 39, 14, 39,

In [27]:
config_file = "train_config.py"
with open(config_file) as f:
    print(f.read())
exec(open(config_file).read())

config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging

# config for training GPT-2 (124M) down to very nice loss of ~2.85 on 1 node of 8X A100 40GB
# launch as the following (e.g. in a screen session) and wait ~5 days:
# $ torchrun --standalone --nproc_per_node=8 train.py config/train_gpt2.py

wandb_log = True
wandb_project = 'gpt_train_string_reversal'
# wandb_run_name=''

# these make the total batch size be ~0.5M
# 12 batch size * 1024 block size * 5 gradaccum * 8 GPUs = 491,520
batch_size = 16
block_size = 20
gradient_accumulation_steps = 5 * 8

# this makes total number of tokens be 300B
max_iters = 600000
lr_decay_iters = 600000

# eval stuff
eval_interval = 1000
eval_iters = 200
log_interval = 10

# weight decay
weight_decay = 1e-1

# Model stuff
n_layer = 2
n_head = 4
n_embd = 32

learning_rate = 1e-4 # max learning rate

max_iters = 600000 # total number of training iterations

beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
decay_lr = True # whether to decay the learning rate
warmup_i

In [28]:
config

{'dtype': 'bfloat16',
 'vocab_size': 41,
 'line': 'alls up theshepherd_drehpehseht pu slla^\n',
 'n': 900000,
 'block_size': 20,
 't': 19,
 'batch_size': 16,
 'config_file': 'train_config.py',
 'wandb_log': True,
 'wandb_project': 'gpt_train_string_reversal',
 'gradient_accumulation_steps': 40,
 'max_iters': 600000,
 'lr_decay_iters': 600000,
 'eval_interval': 1000,
 'eval_iters': 200,
 'log_interval': 10,
 'weight_decay': 0.1,
 'n_layer': 2,
 'n_head': 4,
 'n_embd': 32,
 'learning_rate': 0.0001,
 'beta1': 0.9,
 'beta2': 0.95,
 'grad_clip': 1.0,
 'decay_lr': True,
 'warmup_iters': 2000,
 'bias': False,
 'dropout': True}

In [29]:
# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line
model_args['vocab_size'] = vocab_size  

In [30]:
from gpt.model import GPTConfig
from gpt.model import GPT

In [31]:
gptconf = GPTConfig(**model_args)
model = GPT(gptconf)

number of parameters: 0.03M


In [33]:
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args['block_size'] = block_size # so that the checkpoint will have the right value
model.to(device_type)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(41, 32)
    (wpe): Embedding(20, 32)
    (drop): Dropout(p=True, inplace=False)
    (h): ModuleList(
      (0-1): 2 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=32, out_features=96, bias=False)
          (c_proj): Linear(in_features=32, out_features=32, bias=False)
          (attn_dropout): Dropout(p=True, inplace=False)
          (resid_dropout): Dropout(p=True, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=32, out_features=128, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=128, out_features=32, bias=False)
          (dropout): Dropout(p=True, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=32, out_features=41, bias=False)
)

In [34]:
# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))
init_from = 'scratch'
# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
# if init_from == 'resume':
#     optimizer.load_state_dict(checkpoint['optimizer'])
checkpoint = None # free up memory

num decayed parameter tensors: 10, with 26,528 parameters
num non-decayed parameter tensors: 5, with 160 parameters
using fused AdamW: False


In [35]:
# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    data_dict = {"train" : train_data , "val" : val_data}
    for key in data_dict:
        data = data_dict[key]
        losses = torch.zeros(eval_iters)
        start = random.choice(np.arange(len(data)-eval_iters-1))
        i = 0
        for k in range(start, start + eval_iters):
            X, Y = get_batch(data[k])
            logits, loss = model(X, Y)
            losses[i] = loss.item()
            i+=1
        out[key] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log:
    import wandb
    wandb.init(project=wandb_project, config=config)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sagarjoglekar. Use `wandb login --relogin` to force relogin


In [36]:
# training loop
X, Y = get_batch(train_data[0]) # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model # unwrap DDP container if needed
running_mfu = -1.0
sample_cursor = 1
while sample_cursor < max_iters:

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
                "mfu": running_mfu*100, # convert to percentage
            })
        if losses['val'] < best_val_loss:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        logits, loss = model(X, Y)
        loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        X, Y = get_batch(train_data[sample_cursor])
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()
        sample_cursor+=1
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5: # let the training loop settle a bit
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

step 0: train loss 3.7242, val loss 3.7232
iter 0: loss nan, time 1907.16ms, mfu -100.00%
iter 10: loss nan, time 134.76ms, mfu 0.01%
iter 20: loss nan, time 134.36ms, mfu 0.01%
iter 30: loss nan, time 125.34ms, mfu 0.01%
iter 40: loss nan, time 125.47ms, mfu 0.01%
iter 50: loss nan, time 125.55ms, mfu 0.01%
iter 60: loss nan, time 125.58ms, mfu 0.01%
iter 70: loss nan, time 126.28ms, mfu 0.01%
iter 80: loss nan, time 125.72ms, mfu 0.01%
iter 90: loss nan, time 125.62ms, mfu 0.01%
iter 100: loss nan, time 124.74ms, mfu 0.01%
iter 110: loss nan, time 124.55ms, mfu 0.01%
iter 120: loss nan, time 124.70ms, mfu 0.01%
iter 130: loss nan, time 125.23ms, mfu 0.01%
iter 140: loss nan, time 124.45ms, mfu 0.01%
iter 150: loss nan, time 124.59ms, mfu 0.01%
iter 160: loss nan, time 124.65ms, mfu 0.01%
iter 170: loss nan, time 125.46ms, mfu 0.01%
iter 180: loss nan, time 124.48ms, mfu 0.01%
iter 190: loss nan, time 124.80ms, mfu 0.01%
iter 200: loss nan, time 125.29ms, mfu 0.01%
iter 210: loss nan,

KeyboardInterrupt: 